In [1]:
import os
os.chdir('../../..')

In [2]:
import convokit

In [3]:
from convokit import Corpus, download

In [4]:
corpus = Corpus(filename=download('subreddit-Cornell'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/subreddit-Cornell


In [5]:
corpus.print_summary_stats()

Number of Speakers: 7568
Number of Utterances: 74467
Number of Conversations: 10744


## Bag-of-words prediction for utterances having positive scores

In [6]:
from convokit import BoWClassifier, BoWTransformer

In [7]:
bow_transformer = BoWTransformer(obj_type="utterance")

Initializing default unigram CountVectorizer...Done.


In [8]:
print(next(corpus.iter_utterances()))

Utterance('id': 'nyx4d', 'conversation_id': nyx4d, 'reply-to': None, 'speaker': Speaker('id': reddmau5, 'meta': {'num_posts': 3, 'num_comments': 5}), 'timestamp': 1325452698, 'text': "I was just reading about the Princeton Mic-Check and it's getting [national press](http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html).\n\nI want to get a sense of what people felt like around campus. Anything interesting happen? Anything interesting coming up?", 'meta': {'score': 2, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/', 'author_flair_text': ''})


In [9]:
bow_transformer.fit_transform(corpus)

In [10]:
print(next(corpus.iter_utterances()))

Utterance('id': 'nyx4d', 'conversation_id': nyx4d, 'reply-to': None, 'speaker': Speaker('id': reddmau5, 'meta': {'num_posts': 3, 'num_comments': 5}), 'timestamp': 1325452698, 'text': "I was just reading about the Princeton Mic-Check and it's getting [national press](http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html).\n\nI want to get a sense of what people felt like around campus. Anything interesting happen? Anything interesting coming up?", 'meta': {'score': 2, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/', 'author_flair_text': '', 'bow_vector': <1x9340 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>})


In [11]:
bow_classifier = BoWClassifier(obj_type="utterance", labeller=lambda utt: utt.meta['score'] > 0)

Initializing default classification model (standard scaled logistic regression)


In [12]:
bow_classifier.fit_transform(corpus)

In [13]:
next(corpus.iter_utterances()).meta

{'score': 2,
 'top_level_comment': None,
 'retrieved_on': -1,
 'gilded': -1,
 'gildings': None,
 'subreddit': 'Cornell',
 'stickied': False,
 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/',
 'author_flair_text': '',
 'bow_vector': <1x9340 sparse matrix of type '<class 'numpy.int64'>'
 	with 42 stored elements in Compressed Sparse Row format>,
 'prediction': True,
 'pred_score': 0.9999998491105527}

In [14]:
next(corpus.iter_utterances()).meta['bow_vector'].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [15]:
bow_classifier.summarize(corpus).head()

,prediction,pred_score
id,,
dhhm9sa,True,1.0
dw553ml,True,1.0
dvzmhdx,True,1.0
dvzpp79,True,1.0
dw0imao,True,1.0


In [16]:
corpus.get_utterance('15enm8').text

'One, just to get this out of the way: I\'m only a sophomore in high school. In spite of this, my high school is one of the top public schools in New Jersey (and to put it bluntly it\'s a very affluent area... although I\'m not necessarily affluent like my classmates). The point of telling you guys that is kids start talking about all these amazing schools they want to go to in like eighth grade, so I know quite a bit about colleges. As stated in the title, I really want to go to Cornell, and I just was hoping that some of you guys and girls on here would be awesome enough to give out some SAT scores, ACT scores (if you took them), and extra curricular activities you guys got/did? My unweighted GPA is a 3.8 (weighted is a 4.2), and my first PSAT was an overall 1900, and from taking that I (not to sound cocky here) *know* that I\'m going to get that score up a *lot*. I\'m in all the highest level classes I can be in, and I\'m looking to take multiple AP courses next year (junior). Do yo

In [17]:
bow_classifier.get_coefs(feature_names=bow_transformer.get_vocabulary()).head()

,coef
feat_name,
hotels,1.270001
hbhs,1.115690
engine,1.109702
involves,1.081836
lincoln,1.071464


In [18]:
bow_classifier.get_coefs(feature_names=bow_transformer.get_vocabulary()).tail()

,coef
feat_name,
mahogany,-0.667785
ignoreme,-0.722992
hilton,-0.742234
binary,-0.764383
creation,-0.784593


In [19]:
y_true, y_pred = bow_classifier.get_y_true_pred(corpus)

In [20]:
bow_classifier.base_accuracy(corpus)

0.9279546644822538

In [21]:
bow_classifier.accuracy(corpus)

0.9491452589737737

In [22]:
print(bow_classifier.classification_report(corpus))

              precision    recall  f1-score   support

       False       0.88      0.34      0.49      5365
        True       0.95      1.00      0.97     69102

    accuracy                           0.95     74467
   macro avg       0.91      0.67      0.73     74467
weighted avg       0.95      0.95      0.94     74467



## Bag-of-words prediction for comment thread doubling in length versus staying the same length based on first 5 utterances

In [23]:
top_level_comment_ids = [utt.id for utt in corpus.iter_utterances() if utt.id == utt.meta['top_level_comment']]

In [24]:
corpus.print_summary_stats()

Number of Speakers: 7568
Number of Utterances: 74467
Number of Conversations: 10744


In [25]:
len(top_level_comment_ids)

32893

In [26]:
threads_corpus = corpus.reindex_conversations(new_convo_roots=top_level_comment_ids)


['c3p1rn8', 'c3od15i', 'c3ocsyl', 'c3p8bze', 'c3oyf4d']


In [27]:
threads_corpus.print_summary_stats()

Number of Speakers: 6160
Number of Utterances: 63697
Number of Conversations: 32888


In [28]:
for thread in threads_corpus.iter_conversations():
    thread_len = len(list(thread.iter_utterances()))
    if thread_len == 5:
        thread.meta['thread_doubles'] = False
    elif thread_len >= 10:
        thread.meta['thread_doubles'] = True
    else:
        thread.meta['thread_doubles'] = None

In [29]:
bow_transformer2 = BoWTransformer(obj_type="conversation")

Initializing default unigram CountVectorizer...Done.


In [30]:
bow_transformer2.fit_transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [31]:
bow_classifier2 = BoWClassifier(obj_type="conversation", 
                    labeller=lambda convo: convo.meta['thread_doubles'])

Initializing default classification model (standard scaled logistic regression)


In [32]:
bow_classifier2.fit_transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [33]:
summary = bow_classifier2.summarize(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [34]:
summary.head()

,prediction,pred_score
id,,
cx87pi5,True,1.0
e8p3t2v,True,1.0
dnqc6mc,True,1.0
dxw7g0r,True,1.0
e5626fc,True,1.0


In [35]:
summary.tail()

,prediction,pred_score
id,,
e0iez9l,False,4.201057e-08
cyeq0e8,False,2.781444e-08
dmtcex3,False,2.701564e-08
ck1dyvi,False,1.800621e-08
e6m7j9z,False,2.440828e-10


In [36]:
bow_classifier2.base_accuracy(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

0.6761904761904762

In [37]:
bow_classifier2.accuracy(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

1.0

In [38]:
print(bow_classifier2.classification_report(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00       852
        True       1.00      1.00      1.00       408

    accuracy                           1.00      1260
   macro avg       1.00      1.00      1.00      1260
weighted avg       1.00      1.00      1.00      1260



In [39]:
bow_classifier2.get_coefs(feature_names=bow_transformer2.get_vocabulary()).head(10)

,coef
feat_name,
removed,0.681571
welcome,0.620916
word,0.429793
hour,0.375598
brought,0.359186
profile,0.351558
http,0.351212
head,0.323360
www,0.310268


In [40]:
bow_classifier2.get_coefs(feature_names=bow_transformer2.get_vocabulary()).tail(10)

,coef
feat_name,
tried,-0.265967
desk,-0.269317
internet,-0.269911
long,-0.278180
dean,-0.278975
23,-0.282141
extra,-0.316241
hill,-0.317773
goes,-0.361455


In [41]:
bow_classifier2.confusion_matrix(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

array([[852,   0],
       [  0, 408]])

In [42]:
import pandas as pd

In [43]:
bow_classifier2.evaluate_with_cv(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

Using corpus objects...

Running a cross-validated evaluation...
Done.


array([0.69047619, 0.68650794, 0.64285714, 0.69047619, 0.6984127 ])

In [44]:
bow_classifier2.evaluate_with_train_test_split(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

Using corpus objects...

Running a train-test-split evaluation...
Done.


(0.626984126984127, array([[129,  38],
        [ 56,  29]]))

In [45]:
objs = list(threads_corpus.iter_conversations(selector=lambda convo: convo.meta['thread_doubles'] is not None))

In [46]:
objs = bow_classifier2.transform_objs(objs)

Using input list of corpus objects...



In [47]:
bow_classifier2.summarize_objs(objs)

,prediction,pred_score
id,,
dfyjyd0,False,3.696937e-11
e6m7j9z,False,6.340526e-10
c9txiro,False,1.438202e-08
dvd271f,False,5.431688e-08
df9whw9,False,7.367033e-08
cyeq0e8,False,1.877672e-07
ck1dyvi,False,4.747528e-07
c2kyyy0,False,7.159090e-07
cq9rluh,False,8.290727e-07


## Paired bag-of-words prediction for comment thread doubling in length versus staying the same length based on first 5 utterances

In [48]:
from convokit import Pairer, PairedBoW

In [49]:
corpus = Corpus(filename=download('subreddit-Cornell'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/subreddit-Cornell


In [50]:
threads_corpus = corpus.reindex_conversations(new_convo_roots=top_level_comment_ids, preserve_convo_meta=True)


['c3p1rn8', 'c3od15i', 'c3ocsyl', 'c3p8bze', 'c3oyf4d']


In [51]:
next(threads_corpus.iter_conversations())

Conversation({'obj_type': 'conversation', '_owner': <convokit.model.corpus.Corpus object at 0x13fb89f90>, 'meta': {'original_convo_meta': {'title': 'Arts and Sciences Computer Science Math Courses', 'num_comments': 5, 'domain': 'self.Cornell', 'timestamp': 1467582351, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': None, 'stickied': False, 'author_flair_text': ''}, 'original_convo_id': '4r3zk5'}, '_id': 'd4y41b6', '_utterance_ids': ['d4y41b6'], '_speaker_ids': None, 'tree': None})

In [52]:
for thread in threads_corpus.iter_conversations():
    thread_len = len(list(thread.iter_utterances()))
    if thread_len == 5:
        thread.meta['thread_doubles'] = False
    elif thread_len >= 10:
        thread.meta['thread_doubles'] = True
    else:
        thread.meta['thread_doubles'] = None

In [53]:
next(threads_corpus.iter_conversations()).meta

{'original_convo_meta': {'title': 'Arts and Sciences Computer Science Math Courses',
  'num_comments': 5,
  'domain': 'self.Cornell',
  'timestamp': 1467582351,
  'subreddit': 'Cornell',
  'gilded': 0,
  'gildings': None,
  'stickied': False,
  'author_flair_text': ''},
 'original_convo_id': '4r3zk5',
 'thread_doubles': None}

In [54]:
pairer = Pairer(obj_type="conversation", 
                pos_label_func=lambda convo: convo.meta['thread_doubles'], 
                neg_label_func=lambda convo: convo.meta['thread_doubles'] == False,
                pairing_func=lambda convo: convo.meta['original_convo_id']
               )

In [55]:
pairer.transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [56]:
print(next(threads_corpus.iter_conversations()))

Conversation('id': 'd4y41b6', 'utterances': ['d4y41b6'], 'meta': {'original_convo_meta': {'title': 'Arts and Sciences Computer Science Math Courses', 'num_comments': 5, 'domain': 'self.Cornell', 'timestamp': 1467582351, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': None, 'stickied': False, 'author_flair_text': ''}, 'original_convo_id': '4r3zk5', 'thread_doubles': None, 'pair_obj_label': None, 'pair_id': None, 'pair_orientation': None})


In [57]:
for convo in threads_corpus.iter_conversations():
    if convo.meta['pair_id'] is not None:
        print(convo)
        break

Conversation('id': 'e6m8mw9', 'utterances': ['e6m8mw9', 'e6m8vkh', 'e6m9mnv', 'e6mo8i2', 'e6m9nqu', 'e6movyz', 'e6mb425', 'e6mpmn8', 'e6mr4c8', 'e6mrcet', 'e6na4yk', 'e6nh23h'], 'meta': {'original_convo_meta': {'title': 'Depressed? Anxious? Lonely?', 'num_comments': 39, 'domain': 'self.Cornell', 'timestamp': 1537889370, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': {'gid_1': 0, 'gid_2': 0, 'gid_3': 0}, 'stickied': True, 'author_flair_text': ''}, 'original_convo_id': '9it4aa', 'thread_doubles': True, 'pair_obj_label': 'pos', 'pair_id': '9it4aa', 'pair_orientation': 'pos'})


In [58]:
from convokit import BoWTransformer

In [59]:
bow_transformer = BoWTransformer(obj_type="conversation", 
                                 text_func=lambda convo: " ".join([utt.text for utt in convo.get_chronological_utterance_list()][:5]),
                    )

Initializing default unigram CountVectorizer...Done.


In [60]:
bow_transformer.fit_transform(threads_corpus, selector=lambda convo: convo.meta['pair_id'] is not None)

In [61]:
paired_bow = PairedBoW(obj_type="conversation")

In [62]:
paired_bow.fit(threads_corpus)

Found 65 valid pairs.


In [63]:
paired_bow.summarize(threads_corpus)

Found 65 valid pairs.


0.49230769230769234

In [64]:
paired_bow.get_coefs(feature_names=bow_transformer.get_vocabulary())

,coef
feat_name,
school,0.346201
shit,0.333301
accepted,0.300391
similar,0.290360
computer,0.277568
him,0.253636
since,0.248923
wrong,0.247410
engineering,0.243452
